In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pysandag.database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
sched_dev_parcel_sql = '''
SELECT [site_id]
      ,[parcel_id]
      ,[capacity_3]
      ,[sfu_effective_adj]
      ,[mfu_effective_adj]
      ,[mhu_effective_adj]
      ,[notes]
      ,[editor]
  FROM [urbansim].[urbansim].[scheduled_development_parcel]
'''

sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)

In [ ]:
yr_for_each_site = sched_dev_df.drop_duplicates(subset='site_id',keep='first').copy()

In [ ]:
yr_for_each_site.head()

In [ ]:
len(yr_for_each_site)

In [ ]:
sites = sched_dev_df.site_id.unique()

In [ ]:
len(sites)

In [ ]:
random_yr = np.random.randint(2017,high=2023, size=(len(sites)))

In [ ]:
yr_for_each_site['yr'] = random_yr

In [ ]:
yr_for_each_site.head()

In [ ]:
yr_for_each_site = yr_for_each_site[['site_id','yr']].copy()

In [ ]:
yr_for_each_site.head()

In [ ]:
sched_dev_w_year = pd.merge(sched_dev_df,yr_for_each_site, left_on='site_id',right_on='site_id',how='left')

In [ ]:
sched_dev_w_year.head()

In [ ]:
sched_dev_w_year['sched_version_id'] = 1

In [ ]:
sched_dev_for_db = sched_dev_w_year[['sched_version_id','parcel_id','yr','site_id','capacity_3']].copy()

In [ ]:
len(sched_dev_for_db)

In [ ]:
sched_dev_for_db.head()

In [ ]:
sched_dev_for_db.capacity_3.sum()

In [ ]:
sched_dev_for_db.to_sql(name='scheduled_development_do_not_use', con=mssql_engine, schema='urbansim', index=False,if_exists='replace')